In [ ]:
# 📦 Install Required Packages
%pip install -q google-generativeai kiteconnect transformers torch requests

print("✅ All packages installed successfully!")


In [ ]:
# 🔑 Import Libraries & Load API Keys
import requests
import json
import re
import time
from datetime import datetime

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import google.generativeai as genai
from kiteconnect import KiteConnect
from google.colab import userdata

# Load API keys from Colab secret storage
NEWSDATA_API_KEY = userdata.get("NEWSDATA_API_KEY")
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
ZERODHA_API_KEY = userdata.get("ZERODHA_API_KEY")
ZERODHA_ACCESS_TOKEN = userdata.get("ZERODHA_ACCESS_TOKEN")

# Trading configuration
CAPITAL_PER_TRADE = 25000  # INR - Adjust this amount as needed

# Validate all keys are present
assert all([
    NEWSDATA_API_KEY,
    GEMINI_API_KEY,
    ZERODHA_API_KEY,
    ZERODHA_ACCESS_TOKEN,
]), (
    "❌ One or more API keys missing. Please use userdata.set() to store them first."
)

print("✅ All API keys loaded successfully!")
print(f"💰 Capital per trade set to: ₹{CAPITAL_PER_TRADE:,}")


In [ ]:
# 🤖 Initialize AI Models & Trading Client
print("Loading FinBERT sentiment analysis model...")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
finbert_model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
label_map = finbert_model.config.id2label
print("✅ FinBERT model loaded successfully!")

print("Configuring Gemini AI for ticker extraction...")
genai.configure(api_key=GEMINI_API_KEY)
gemini_model = genai.GenerativeModel("gemini-2.0-flash")
print("✅ Gemini AI configured successfully!")

print("Initializing Zerodha KiteConnect client...")
kite = KiteConnect(api_key=ZERODHA_API_KEY)
kite.set_access_token(ZERODHA_ACCESS_TOKEN)
print("✅ KiteConnect client initialized successfully!")

print("\n🎯 All systems ready for trading!")


In [ ]:
# 🛠️ Helper Functions

def log(msg: str) -> None:
    """Time-stamped console logger for audit trail."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{timestamp}] - {msg}")


def get_sentiment(text: str):
    """Analyze sentiment using FinBERT model."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = finbert_model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        score, idx = torch.max(probs, dim=1)
    return label_map[idx.item()], score.item()


ticker_cache = {}

def extract_ticker(headline: str):
    """Use Gemini AI to extract company name and NSE ticker from news headline."""
    cache_key = headline.lower().strip()
    if cache_key in ticker_cache:
        return ticker_cache[cache_key]

    prompt = (
        "You are a seasoned financial analyst specializing in Indian stock markets.\n\n"
        f'Headline: "{headline}"\n\n'
        "Return ONLY valid JSON:\n"
        '{"company_name": "Full Official Company Name", "ticker_symbol": "NSE_SYMBOL"}'
    )

    try:
        response = gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.1,
                max_output_tokens=120,
            ),
        )
        text = response.text.strip()
        text = re.sub(r"^```json|```$", "", text).strip()
        data = json.loads(text)
        
        company_name = data["company_name"]
        ticker_symbol = data["ticker_symbol"]
        
        # Cache the result
        ticker_cache[cache_key] = (company_name, ticker_symbol)
        return company_name, ticker_symbol
        
    except Exception as exc:
        log(f"❌ Gemini error: {exc}")
        return None, None


def fetch_recent_headlines(limit: int = 15):
    """Fetch latest Indian financial news headlines."""
    url = "https://newsdata.io/api/1/latest"
    params = {
        "apikey": NEWSDATA_API_KEY,
        "q": "finance OR stock OR IPO OR investment OR market OR NSE OR BSE",
        "country": "in",
        "language": "en",
        "category": "business",
    }

    try:
        response = requests.get(url, params=params, timeout=20)
        data = response.json()
        
        if data.get("status") != "success":
            log(f"⚠️ NewsData error: {data.get('message')}")
            return []
            
        articles = [article for article in data.get("results", []) if "title" in article]
        articles.sort(key=lambda x: x.get("pubDate", ""), reverse=True)
        
        headlines = [
            {"title": article["title"], "published": article.get("pubDate")}
            for article in articles
        ][:limit]
        
        log(f"✅ Fetched {len(headlines)} recent headlines")
        return headlines
        
    except Exception as exc:
        log(f"❌ Error fetching news: {exc}")
        return []

print("✅ Helper functions defined successfully!")


In [ ]:
# 🚀 Main Trading Bot Function

def run_trading_bot():
    """
    Main function that orchestrates the trading process:
    1. Fetch recent financial news headlines
    2. Extract tickers using AI
    3. Analyze sentiment
    4. Execute trades based on criteria
    """
    log("=" * 90)
    log("🚀 STARTING AUTOMATED TRADING BOT")
    log("=" * 90)

    # Fetch latest headlines
    headlines = fetch_recent_headlines()
    if not headlines:
        log("❌ No headlines found. Exiting.")
        return

    log(f"📰 Processing {len(headlines)} headlines...")
    
    for idx, item in enumerate(headlines, 1):
        headline = item["title"]
        log(f"\n[{idx}/{len(headlines)}] NEWS: \"{headline}\"")

        # Step 1: Extract ticker using AI
        company, symbol = extract_ticker(headline)
        if not (company and symbol):
            log("⚠️ Ticker not identified. Skipping.")
            continue

        log(f"🏢 COMPANY: {company} ({symbol})")

        # Step 2: Analyze sentiment
        sentiment, confidence = get_sentiment(headline)
        log(f"📊 SENTIMENT: {sentiment} (confidence={confidence:.3f})")

        # Step 3: Check trading criteria
        if sentiment.lower() != "positive" or confidence <= 0.8:
            log(f"❌ Criteria not met (need: positive sentiment & confidence > 0.8). Skipping trade.")
            continue

        log("✅ Trading criteria met! Proceeding with order...")

        # Step 4: Get live stock price
        try:
            ltp_data = kite.ltp(f"NSE:{symbol}")
            live_price = ltp_data[f"NSE:{symbol}"]["last_price"]
            log(f"💰 Live price for {symbol}: ₹{live_price}")
        except Exception as exc:
            log(f"❌ Failed to fetch live price: {exc}")
            continue

        # Step 5: Check capital sufficiency
        if CAPITAL_PER_TRADE <= live_price:
            log(f"❌ Capital per trade (₹{CAPITAL_PER_TRADE:,}) insufficient for single share (₹{live_price}). Skipping.")
            continue

        # Step 6: Calculate quantity
        quantity = int(CAPITAL_PER_TRADE / live_price)
        if quantity <= 0:
            log("❌ Calculated quantity is 0. Skipping.")
            continue

        log(f"📈 Planning to buy {quantity} shares (₹{quantity * live_price:,.2f} total)")

        # Step 7: Place BUY order
        try:
            order_id = kite.place_order(
                variety=kite.VARIETY_REGULAR,
                exchange=kite.EXCHANGE_NSE,
                tradingsymbol=symbol,
                transaction_type=kite.TRANSACTION_TYPE_BUY,
                quantity=quantity,
                product=kite.PRODUCT_CNC,
                order_type=kite.ORDER_TYPE_MARKET,
            )
            log(f"🎉 ORDER EXECUTED: Bought {quantity} shares of {symbol} at market price. Order ID: {order_id}")
            
        except Exception as exc:
            log(f"❌ Order placement failed: {exc}")

        # Brief pause between processing headlines
        time.sleep(0.5)

    log("\n" + "=" * 90)
    log("✅ Completed processing all headlines.")
    log("=" * 90)

print("✅ Main trading function defined successfully!")


In [ ]:
# ▶️ RUN THE TRADING BOT

# Execute the trading bot
run_trading_bot()
